In [ ]:
# IMPORTS

import os 
import time
from io import StringIO
import numpy as np
import pandas as pd

import openpyxl

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from numpy.polynomial.polynomial import Polynomial
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error


import jou_gen as jg

import ansys.fluent.core as pyfluent

In [ ]:
mdot_distrib_inlet_param = 0.0278 # kg/s

In [ ]:
def process(distrib_df, coll_df, gamma=0.1):

    distrib_df['mdot'] = distrib_df['mdot'].where(distrib_df['mdot'] > 0, np.nan)
    distrib_df['distance'] = (distrib_df['mdot'] - coll_df['mdot'])/coll_df['mdot']

    distrib_df['mdot_corrected'] = (1+gamma*distrib_df['distance'])*coll_df['mdot']
    distrib_df['mdot_corrected'] = distrib_df['mdot_corrected'].fillna(coll_df['mdot'])

    sum_mdot_corrected = distrib_df['mdot_corrected'].sum()
    distrib_df['mdot_corrected'] *= mdot_distrib_inlet_param/sum_mdot_corrected

In [ ]:
def process_v2(distrib_df, coll_df, gamma=0.1):

    distrib_df['mdot'] = distrib_df['mdot'].where(distrib_df['mdot'] > 0, np.nan)
    distrib_df['distance'] = (distrib_df['mdot'] - coll_df['mdot'])/coll_df['mdot']
    rmse = mean_squared_error(distrib_df['mdot'], coll_df['mdot'], squared=False)
    if rmse/coll_df['mdot'].mean() < 1:
        gamma = 1 - rmse/coll_df['mdot'].mean()
    else:
        pass

    distrib_df['mdot_corrected'] = (1+gamma*distrib_df['distance'])*coll_df['mdot']
    nan_count = distrib_df['mdot_corrected'].isna().sum()
    sum_mdot_corrected = distrib_df['mdot_corrected'].sum()
    distrib_df['mdot_corrected'] = distrib_df['mdot_corrected'].fillna((mdot_distrib_inlet_param-sum_mdot_corrected)/nan_count)

    # Get the x and y values for interpolation
    x = distrib_df.index
    y = distrib_df['mdot_corrected']

    # Fit the polynomial of degree 2
    poly = Polynomial.fit(x, y, deg=2)

    # Generate the interpolated values
    interpolated_values = poly(x)

    distrib_df['mdot_corrected'] = interpolated_values

    distrib_df.loc[distrib_df['mdot_corrected'] < 0, 'mdot_corrected'] = np.nan
    nan_count = distrib_df['mdot_corrected'].isna().sum()
    sum_mdot_corrected = distrib_df['mdot_corrected'].sum()
    distrib_df['mdot_corrected'] = distrib_df['mdot_corrected'].fillna((mdot_distrib_inlet_param-sum_mdot_corrected)/nan_count)

    sum_mdot_corrected = distrib_df['mdot_corrected'].sum()
    distrib_df['mdot_corrected'] = distrib_df['mdot_corrected'] * (mdot_distrib_inlet_param/sum_mdot_corrected)

    return distrib_df

In [ ]:
fp_temp = r'D:\ANSYS Fluent Projects\temp'

# folder_path = r'D:\ANSYS Fluent Projects\Hydraulics\V4.5\19MPE_woch\fluent'
folder_path = r'D:\ANSYS Fluent Projects\Hydraulics\V4.5\19MPE_woch\fluent\V3'

N_junction = 38
N_channels = 304
Area_junction = 44.0414E-6

N_it = 10
N_step1 = 12
N_big_it = 60

In [ ]:
solver_path = folder_path + '\\' + 'server_info-19460.txt'
solver = pyfluent.connect_to_fluent(server_info_filepath=solver_path)
tui = solver.tui

options for try2 : prevent reverse flow everywhere and pressure outlet is calculated with regular pressure loss with the mdot in collector

### STEP 1

In [ ]:
# # Initialize velocities with uniform flows

# string_list = []
# for i in range(N_junction):
#     string_list.append(jg.change_bc_velocity_inlet(f"coll_ch_{i+1}",f"((mdot_distrib_inlet/rho_water)/{N_junction})/({Area_junction} [m^2])"))
# jg.concatenate_and_write_to_file(string_list,fp_temp + '\\change_bc_v_uniform.txt')
# tui.file.read_journal(f'"{fp_temp}\\change_bc_v_uniform.txt"')

In [ ]:
# Initialize with uniform mass flows

string_list = []
for i in range(N_junction):
    string_list.append(jg.change_bc_mdot_inlet(f"coll_ch_{i+1}",f"(mdot_distrib_inlet/{N_junction})"))
jg.concatenate_and_write_to_file(string_list,fp_temp + '\\change_bc_v_uniform.txt')
tui.file.read_journal(f'"{fp_temp}\\change_bc_v_uniform.txt"')

In [ ]:
# # Initialize with previous flow distribution

# distrib_df, coll_df = export(folder_path, 110)
# process(distrib_df, coll_df, gamma=0.3)

# string_list = []
# for i in range(N_junction):
#     string_list.append(jg.change_bc_velocity_inlet(f"coll_ch_{i+1}",f"( {distrib_df.loc[i+1,'mdot_corrected']} [kg/s] / rho_water ) / (44.0414E-6 [m^2])"))
# jg.concatenate_and_write_to_file(string_list,fp_temp+'\\change_bc_v.txt')
# tui.file.read_journal(f'"{fp_temp}\\change_bc_v.txt"')

In [ ]:
# # Regular pressure loss depend on mdot_distrib_inlet/304 (uniformity assumption for regular pressure losses)

# string_list = []
# for i in range(N_junction):
#     string_list.append(jg.change_bc_pressure_outlet(name=f"distrib_ch_{i+1}",
#                                                     value=f"{{sp_coll_ch_{i+1}}} + 1411785 [Pa s / kg] * ( mdot_distrib_inlet / 304)",
#                                                     backflow_direction_specification_method='prevent backflow'))
# jg.concatenate_and_write_to_file(string_list,fp_temp+'\\change_bc_po_uniform_prevent_backflow.txt')

# tui.file.read_journal(f'"{fp_temp}\\change_bc_po_uniform_prevent_backflow.txt"')

In [ ]:
# Regular pressure loss depend on pressure in coll_ch with target mass flow rate (test without preventing backflow)

string_list = []
for i in range(N_junction):
    string_list.append(jg.change_bc_pressure_outlet(name=f"distrib_ch_{i+1}",
                                                    value=f"{{sp_coll_ch_{i+1}}} + 1411785 [Pa s / kg] * ( {{mdot_coll_ch_{i+1}}} / 8)",
                                                    backflow_direction_specification_method='prevent backflow',
                                                    target_mass_flow_rate=True,
                                                    mdot=f"(mdot_distrib_inlet/ {N_junction} )")
    )
jg.concatenate_and_write_to_file(string_list,fp_temp+'\\change_bc_po_uniform_with_backflow_target.txt')
tui.file.read_journal(f'"{fp_temp}\\change_bc_po_uniform_with_backflow_target.txt"')

In [ ]:
solver.solution.initialization.hybrid_initialize()
solver.solution.run_calculation.iterate(number_of_iterations=100)

In [ ]:
IT = 0

jg.write_report(tui, 'mdot', folder_path, f'mdot_report_IT{IT}')
jg.write_report(tui, 'sp', folder_path, f'sp_report_IT{IT}')

In [ ]:
distrib_df, coll_df = jg.export(folder_path, f'IT{IT}')

In [ ]:
process(distrib_df, coll_df, gamma=0.3)

In [ ]:
distrib_df['mdot'].sum()

In [ ]:
fig, ax = plt.subplots()

ax.scatter(distrib_df.index, distrib_df['mdot'], label='mdot distrib', marker='+')
ax.scatter(distrib_df.index, distrib_df['mdot_corrected'], label='mdot distrib corrected')
ax.scatter(coll_df.index, coll_df['mdot'], label='mdot coll',marker='+')
ax.legend()

In [ ]:
# Adapt on the fly

string_list = []
for i in range(N_junction):
    string_list.append(jg.change_bc_mdot_inlet(f"coll_ch_{i+1}",distrib_df.loc[i+1,'mdot_corrected']))
jg.concatenate_and_write_to_file(string_list,fp_temp + '\\change_bc_v_adapted.txt')
tui.file.read_journal(f'"{fp_temp}\\change_bc_v_adapted.txt"')

In [ ]:
# Regular pressure loss depend on pressure in coll_ch with target mass flow rate (test without preventing backflow)

string_list = []
for i in range(N_junction):
    string_list.append(jg.change_bc_pressure_outlet(name=f"distrib_ch_{i+1}",
                                                    value=f"{{sp_coll_ch_{i+1}}} + 1411785 [Pa s / kg] * ( {{mdot_coll_ch_{i+1}}} / 8)",
                                                    backflow_direction_specification_method='Normal to Boundary',
                                                    target_mass_flow_rate=True,
                                                    mdot=distrib_df.loc[i+1,'mdot_corrected'])
    )
jg.concatenate_and_write_to_file(string_list,fp_temp+'\\change_bc_po_uniform_with_backflow_target.txt')
tui.file.read_journal(f'"{fp_temp}\\change_bc_po_uniform_with_backflow_target.txt"')

In [ ]:
start_time = time.perf_counter()

for i in range(0,N_step1):
    IT = 10*i
    print('BIG ITERATION N°:',IT)

    print('Start 10 iterations')
    solver.solution.run_calculation.iterate(number_of_iterations=N_it)
    print('10 iterations done')

    jg.write_report(tui, 'mdot', folder_path, f'mdot_report_IT{IT}')
    jg.write_report(tui, 'sp', folder_path, f'sp_report_IT{IT}')
    print('Reports written')

    distrib_df, coll_df = export(folder_path, IT)
    process(distrib_df, coll_df, gamma=0.3)
    print('Data processed')

    fig, ax = plt.subplots()

    ax.scatter(distrib_df.index, distrib_df['mdot'], label='mdot distrib', marker='+')
    ax.scatter(distrib_df.index, distrib_df['mdot_corrected'], label='mdot distrib corrected')
    ax.scatter(coll_df.index, coll_df['mdot'], label='mdot coll',marker='+')

    plt.legend()
    plt.savefig(folder_path + f'\\fig_mdot_{IT}.png')
    plt.close(fig)
    print('Fig saved')

    string_list = []
    for i in range(N_junction):
        string_list.append(jg.change_bc_velocity_inlet(f"coll_ch_{i+1}",f"( {distrib_df.loc[i+1,'mdot_corrected']} [kg/s] / rho_water ) / (44.0414E-6 [m^2])"))
    jg.concatenate_and_write_to_file(string_list,fp_temp+'\\change_bc_v.txt')
    tui.file.read_journal(f'"{fp_temp}\\change_bc_v.txt"')

end_time = time.perf_counter()

print(f'Elapsed time: {end_time - start_time} seconds')

### STEP 2

In [ ]:
def update_bc_po_connected(i):
    string_list = []
    string_list.append(jg.change_bc_pressure_outlet(name=f"distrib_ch_{i}",
                                                        value=f"{{sp_coll_ch_{i}}} + 1411785 [Pa s / kg] * ( {{mdot_coll_ch_{i}}} / 8)",
                                                        backflow_direction_specification_method='prevent backflow'))
    jg.concatenate_and_write_to_file(string_list,fp_temp+'\\change_bc_po_non_uniform_prevent_backflow.txt')
    tui.file.read_journal(f'"{fp_temp}\\change_bc_po_non_uniform_prevent_backflow.txt"')

In [ ]:
mdot_distrib_inlet_param/N_junction

In [ ]:
# Regular pressure loss depend on mdot in collector now

string_list = []
for i in range(0,N_junction):
    string_list.append(jg.change_bc_pressure_outlet(name=f"distrib_ch_{i+1}",
                                                    value=f"{{sp_coll_ch_{i+1}}} + 1411785 [Pa s / kg] * ( {{mdot_coll_ch_{i+1}}} / 8)",
                                                    backflow_direction_specification_method='prevent backflow',
                                                    target_mass_flow_rate = True,
                                                    mdot = mdot_distrib_inlet_param/N_junction,
                                                    pressure_upper_bound = 300,
                                                    pressure_lower_bound = 20,
                                                    ))
jg.concatenate_and_write_to_file(string_list,fp_temp+'\\change_bc_po_non_uniform_prevent_backflow.txt')
tui.file.read_journal(f'"{fp_temp}\\change_bc_po_non_uniform_prevent_backflow.txt"')

In [ ]:
N_step1 = 10

In [ ]:
update_bc_po_connected(11)
update_bc_po_connected(12)
update_bc_po_connected(13)
update_bc_po_connected(14)
update_bc_po_connected(15)
update_bc_po_connected(16)

In [ ]:
start_time = time.perf_counter()

for i in range(N_step1+22,N_step1+35):
    IT = 10*i
    print('BIG ITERATION N°:',IT)

    print('Start 10 iterations')
    solver.solution.run_calculation.iterate(number_of_iterations=N_it)
    print('10 iterations done')

    jg.write_report(tui, 'mdot', folder_path, f'mdot_report_IT{IT}')
    jg.write_report(tui, 'sp', folder_path, f'sp_report_IT{IT}')
    print('Reports written')

    distrib_df, coll_df = export(folder_path, IT)
    process(distrib_df, coll_df, gamma=0.3)
    print('Data processed')

    fig, ax = plt.subplots()

    ax.scatter(distrib_df.index, distrib_df['mdot'], label='mdot distrib', marker='+')
    ax.scatter(distrib_df.index, distrib_df['mdot_corrected'], label='mdot distrib corrected')
    ax.scatter(coll_df.index, coll_df['mdot'], label='mdot coll',marker='+')

    plt.legend()
    plt.savefig(folder_path + f'\\fig_mdot_{IT}.png')
    plt.close(fig)
    print('Fig saved')

    string_list = []
    for i in range(N_junction):
        string_list.append(jg.change_bc_velocity_inlet(f"coll_ch_{i+1}",f"( {distrib_df.loc[i+1,'mdot_corrected']} [kg/s] / rho_water ) / (44.0414E-6 [m^2])"))
    jg.concatenate_and_write_to_file(string_list,fp_temp+'\\change_bc_v.txt')
    tui.file.read_journal(f'"{fp_temp}\\change_bc_v.txt"')

end_time = time.perf_counter()

print(f'Elapsed time: {end_time - start_time} seconds')

In [ ]:
# STEP 2 MODIFIED

start_time = time.perf_counter()

for i in range(N_step1,N_big_it):
    IT = 10*i
    print('BIG ITERATION N°:',IT)

    if i == N_step1:
        pass
    else:
        print('Start 10 iterations')
        solver.solution.run_calculation.iterate(number_of_iterations=N_it)
        print('10 iterations done')

    jg.write_report(tui, 'mdot', folder_path, f'mdot_report_IT{IT}')
    jg.write_report(tui, 'sp', folder_path, f'sp_report_IT{IT}')
    print('Reports written')

    distrib_df, coll_df = export(folder_path, IT)
    process_v2(distrib_df, coll_df, gamma=0.3)
    print('Data processed')

    fig, ax = plt.subplots()

    ax.scatter(distrib_df.index, distrib_df['mdot'], label='mdot distrib', marker='+')
    ax.scatter(distrib_df.index, distrib_df['mdot_corrected'], label='mdot distrib corrected')
    ax.scatter(coll_df.index, coll_df['mdot'], label='mdot coll',marker='+')

    plt.legend()
    plt.savefig(folder_path + f'\\fig_mdot_{IT}.png')
    plt.close(fig)
    print('Fig saved')

    string_list = []
    for i in range(N_junction):
        string_list.append(jg.change_bc_velocity_inlet(f"coll_ch_{i+1}",f"( {distrib_df.loc[i+1,'mdot_corrected']} [kg/s] / rho_water ) / (44.0414E-6 [m^2])"))
    jg.concatenate_and_write_to_file(string_list,fp_temp+'\\change_bc_v.txt')
    tui.file.read_journal(f'"{fp_temp}\\change_bc_v.txt"')
    
    ### With target mass flow
    string_list = []
    for i in range(N_junction):
        # sp_value = coll_df.loc[i+1,'sp'] + 1411785 * (distrib_df.loc[i+1,'new_input'] / 8)
        string_list.append(jg.change_bc_pressure_outlet(name=f"distrib_ch_{i+1}",
                                                            value = f"{{sp_coll_ch_{i+1}}} + 1411785 [Pa s / kg] * ( {{mdot_coll_ch_{i+1}}} / 8)",
                                                            backflow_direction_specification_method = 'prevent backflow',
                                                            target_mass_flow_rate = True,
                                                            mdot = distrib_df.loc[i+1,'mdot_corrected'],
                                                            pressure_upper_bound = 1.5 * ( coll_df['sp'].loc[i+1] + 1411785 * (distrib_df['mdot_corrected'].loc[i+1] / 8) ),
                                                            pressure_lower_bound= 0.5 * ( coll_df['sp'].loc[i+1] + 1411785 * (distrib_df['mdot_corrected'].loc[i+1] / 8) )
                                                            ))
    jg.concatenate_and_write_to_file(string_list,fp_temp + '\\change_bc_po_non_uniform_target.txt')
    tui.file.read_journal(f'"{fp_temp}\\change_bc_po_non_uniform_target.txt"')

    

end_time = time.perf_counter()

print(f'Elapsed time: {end_time - start_time} seconds')

In [ ]:
start_time = time.perf_counter()

for i in range(N_step1,N_big_it):
    IT = 10*i
    print('BIG ITERATION N°:',IT)

    if i == N_step1:
        pass
    else:
        print('Start 10 iterations')
        solver.solution.run_calculation.iterate(number_of_iterations=N_it)
        print('10 iterations done')

    jg.write_report(tui, 'mdot', folder_path, f'mdot_report_IT{IT}')
    jg.write_report(tui, 'sp', folder_path, f'sp_report_IT{IT}')
    print('Reports written')

    distrib_df, coll_df = export(folder_path, IT)
    process_v2(distrib_df, coll_df, gamma=0.3)
    print('Data processed')

    fig, ax = plt.subplots()

    ax.scatter(distrib_df.index, distrib_df['mdot'], label='mdot distrib', marker='+')
    ax.scatter(distrib_df.index, distrib_df['mdot_corrected'], label='mdot distrib corrected')
    ax.scatter(coll_df.index, coll_df['mdot'], label='mdot coll',marker='+')

    plt.legend()
    plt.savefig(folder_path + f'\\fig_mdot_{IT}.png')
    plt.close(fig)
    print('Fig saved')

    string_list = []
    for i in range(N_junction):
        string_list.append(jg.change_bc_velocity_inlet(f"coll_ch_{i+1}",f"( {distrib_df.loc[i+1,'mdot_corrected']} [kg/s] / rho_water ) / (44.0414E-6 [m^2])"))
    jg.concatenate_and_write_to_file(string_list,fp_temp+'\\change_bc_v.txt')
    tui.file.read_journal(f'"{fp_temp}\\change_bc_v.txt"')
    
    ### With target mass flow
    string_list = []
    for i in range(N_junction):
        # sp_value = coll_df.loc[i+1,'sp'] + 1411785 * (distrib_df.loc[i+1,'new_input'] / 8)
        string_list.append(jg.change_bc_pressure_outlet(name=f"distrib_ch_{i+1}",
                                                            value = f"{{sp_coll_ch_{i+1}}} + 1411785 [Pa s / kg] * ( {{mdot_coll_ch_{i+1}}} / 8)",
                                                            backflow_direction_specification_method = 'prevent backflow',
                                                            target_mass_flow_rate = True,
                                                            mdot = distrib_df.loc[i+1,'mdot_corrected'],
                                                            pressure_upper_bound = 1.5 * ( coll_df['sp'].loc[i+1] + 1411785 * (distrib_df['mdot_corrected'].loc[i+1] / 8) ),
                                                            pressure_lower_bound= 0.5 * ( coll_df['sp'].loc[i+1] + 1411785 * (distrib_df['mdot_corrected'].loc[i+1] / 8) )
                                                            ))
    jg.concatenate_and_write_to_file(string_list,fp_temp + '\\change_bc_po_non_uniform_target.txt')
    tui.file.read_journal(f'"{fp_temp}\\change_bc_po_non_uniform_target.txt"')

    

end_time = time.perf_counter()

print(f'Elapsed time: {end_time - start_time} seconds')

In [ ]:
fig, ax = plt.subplots()

ax.scatter(distrib_df.index, distrib_df['mdot'], label='mdot distrib', marker='+')
# ax.scatter(distrib_df.index, distrib_df['mdot_corrected'], label='mdot distrib corrected')
ax.scatter(coll_df.index, coll_df['mdot'], label='mdot coll',marker='+')

plt.ylim(0, 0.002)
plt.legend()

In [ ]:
string_list = []
for i in range(N_junction):
    string_list.append(jg.change_bc_velocity_inlet(f"coll_ch_{i+1}",f"( {coll_df.loc[i+1,'mdot']} [kg/s] / rho_water ) / (44.0414E-6 [m^2])"))
jg.concatenate_and_write_to_file(string_list,fp_temp+'\\change_bc_v.txt')
tui.file.read_journal(f'"{fp_temp}\\change_bc_v.txt"')

In [ ]:
jg.write_report(tui, 'mdot', folder_path, f'mdot_report_IT{IT}')
jg.write_report(tui, 'sp', folder_path, f'sp_report_IT{IT}')
print('Reports written')

In [ ]:
distrib_df, coll_df = export(folder_path, IT)